
<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to-TFlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/5_Train_Export_to-TFlite.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>



# Tensorflow Lite Conversion #

<img src="https://miro.medium.com/max/128/1*cKG1LJvVTaWqSkYSyVqtsQ.png"  alt="Visit my Blog"> 

<img src="https://lh3.googleusercontent.com/vvBAqSnXyg3h9yS0JLyVehhV-e__3NFbZ6q7Ft-rEZp-9wDTVZ49yjuYJwfa4jQZ-RVnChHMr-DDC0T_fTxVyQg3iBMD-icMQooD6A=w630-rw" alt="Visit my Blog">

Once we have our trained model, there is one final step we must take before we are ready to start testing the model. 

We have to convert the frozen graph into a TF Lite model which our inference application can use. The model is quantized, which means it is compatible with Edge TPU devices such as Google Coral.

### Install required libraries

In [1]:
%tensorflow_version 1.x

!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

!sudo apt-get update

!sudo apt-get install edgetpu-compiler



TensorFlow 1.x selected.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  15547      0 --:--:-- --:--:-- --:--:-- 15547
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Get:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 Packages [37.4 kB]
Ign:9 https://developer.download.n

### Prepare dataset ###

#### Connect to Google Drive

In [2]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Copy dataset from Google Drive and extract

In [3]:
%cd /content/drive/My Drive/Datasets/
%cp model_graph.zip /content/
%cd /content/
!unzip -o /content/model_graph.zip


/content/drive/My Drive/Datasets
/
Archive:  /content/model_graph.zip
   creating: trained_inference_graphs/
  inflating: trained_inference_graphs/checkpoint  
  inflating: trained_inference_graphs/pipeline.config  
  inflating: trained_inference_graphs/model.ckpt.meta  
  inflating: trained_inference_graphs/model.ckpt.data-00000-of-00001  
   creating: trained_inference_graphs/saved_model/
   creating: trained_inference_graphs/saved_model/variables/
  inflating: trained_inference_graphs/saved_model/saved_model.pb  
  inflating: trained_inference_graphs/model.ckpt.index  
  inflating: trained_inference_graphs/frozen_inference_graph.pb  


In [5]:
%cd /content/
!unzip -o /content/model_graph.zip

/content
Archive:  /content/model_graph.zip
   creating: trained_inference_graphs/
  inflating: trained_inference_graphs/checkpoint  
  inflating: trained_inference_graphs/pipeline.config  
  inflating: trained_inference_graphs/model.ckpt.meta  
  inflating: trained_inference_graphs/model.ckpt.data-00000-of-00001  
   creating: trained_inference_graphs/saved_model/
   creating: trained_inference_graphs/saved_model/variables/
  inflating: trained_inference_graphs/saved_model/saved_model.pb  
  inflating: trained_inference_graphs/model.ckpt.index  
  inflating: trained_inference_graphs/frozen_inference_graph.pb  


In [10]:
!tflite_convert \
! --output_file="/content/output_tflite_graph.tflite" \
! --graph_def_file="/content/trained_inference_graphs/frozen_inference_graph.pb" \
! --input_arrays='input' \
! --output_arrays='TFLite_Detection_PostProcess' \
! --inference_type=QUANTIZED_UINT8 \
! --mean_values=128 \
! --std_dev_values=128 \
! --input_shapes=1,300,300,3 \
! --change_concat_input_ranges=false \
! --allow_nudging_weights_to_use_fast_gemm_kernel=true \
! --allow_custom_ops

2020-06-09 16:30:12.775293: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-09 16:30:12.777095: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-06-09 16:30:12.777140: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b2ec1e9bfca0): /proc/driver/nvidia/version does not exist
2020-06-09 16:30:12.777502: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2020-06-09 16:30:12.783186: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000165000 Hz
2020-06-09 16:30:12.783450: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1894a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-06-09 16:30:1

## Save 

### Archive the inference graph

/content/training_demo
--2020-06-05 08:59:06--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/ssd_mobilenet_v2_coco.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4766 (4.7K) [text/plain]
Saving to: ‘ssd_mobilenet_v2_coco.config’

ssd_mobilenet_v2_co 100%[===================>]   4.65K  --.-KB/s    in 0s      

2020-06-05 08:59:06 (46.5 MB/s) - ‘ssd_mobilenet_v2_coco.config’ saved [4766/4766]



### Save archive to Google Drive

/content/training_demo/images/test
185
